In [7]:
import pandas as pd
import csv
from bs4 import BeautifulSoup
import requests
from os import walk, makedirs
import time
import random
import zipfile
import json
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [24]:
gene_list = []

gene_list = pd.read_csv("./dataset/to_scrape.txt", sep=', ')

gene_list = list(gene_list.columns)

fantom_db = pd.read_csv("./dataset/anno-fantom.csv")
snoDB = pd.read_csv("./dataset/snoDB_All_V2.0.csv",keep_default_na=False)

gene_new = {"SNORA11", "SNORA74A", "SNORA63C", "SNORD50B", "SNORD89", "SNORA74A", "SNORD57", "SNORD7", "SNORA31", "SNORD65", "SNORD6", "SNORA25", "SNORA32", "SNORA51", "SNORD8", "SNORA63", "SNORA4", "SNORA66", "SNORA20", "SNORA20", "SNORA67", "SNORD22", "SNORD58C", "SNORD108"}

gene_list = set(gene_list).intersection(gene_new)

In [23]:
def get_gene_links(gene_list):
    url = 'https://gene.disi.unitn.it/test/gene_history-z.php'
    data = {
        'org': 'Hs+(Homo+sapiens)',
        'tsize': '',
        'iter': '',
        'alpha': '0.05',
        'lgn': '',
        'submit': 'SEARCH'}
    missing_list = []

    for gene in gene_list:
        print(f"Looking for {gene}")
        data['lgn'] = gene
        soup = BeautifulSoup(requests.post(url, data = data).text, "lxml")

        if soup.find("div",{"id":"dialog-end"}) != None:
            missing_list.append(gene)
        else:
            table = soup.find("tbody")
            rows = table.find_all("tr")
            for row in rows:
                columns = row.find_all("td")
                if columns[2].string.split("-")[1] == gene:
                    downloaded_file = requests.get(f"https://gene.disi.unitn.it/test/gene_result-z.php?id={columns[0].string}_{columns[1].string}", allow_redirects=True)
                    open(f'./gene_expansions_top_20_sno/{columns[0].string}_{columns[1].string}.zip', 'wb').write(downloaded_file.content) 
        time.sleep(random.uniform(1.5,2.0))

    return missing_list

makedirs("./gene_expansions_top_20_sno", exist_ok=True)
missing_list = get_gene_links(gene_list)
print(f"Missing genes: {', '.join(missing_list)}")

Looking for SNORD65
Looking for SNORA32
Looking for SNORD108
Looking for SNORD89
Looking for SNORD58C
Looking for SNORD57
Looking for SNORA63
Looking for SNORD22
Looking for SNORD8
Missing genes: 
